In [17]:
from scipy.io import loadmat
import json
import numpy
from sklearn.model_selection import train_test_split
annots = loadmat('Xtrain.mat')

In [18]:
traindata=annots['Xtrain']
traindata=numpy.concatenate(traindata, axis=0 )
X_train, X_test= train_test_split(traindata, 
                                   random_state=104,  
                                   test_size=0.20,  
                                   shuffle=True)

In [19]:
print(len(X_train))

800
